In [ ]:
import numpy as np
import torch

from neurodifflogic.difflogic.compiled_model import CompiledCombinedLogicNet

from neurodifflogic.models import CNN

In [2]:
model = CNN(class_count=10, tau=1, implementation="python", device="cpu")
model.train(False)  # Switch model to eval mode

compiled_model = CompiledCombinedLogicNet(
    model=model.model, num_bits=8, cpu_compiler="gcc", verbose=True
)
compiled_model.compile(save_lib_path="compiled_clgn_model.so", verbose=False)


Found Flatten layer
Found GroupSum layer with 10 classes
Parsed 3 conv, 3 pooling, 3 linear layers
Layer execution order: [('conv', 0), ('pool', 0), ('conv', 1), ('pool', 1), ('conv', 2), ('pool', 2), ('flatten', 0), ('linear', 0), ('linear', 1), ('linear', 2)]
Compiling finished in 3.017 seconds.


In [3]:
x = torch.randint(0, 2, (8, 1, 28, 28), dtype=torch.int16)
x_np = x.bool().numpy()

preds = model(x)
preds_compiled = compiled_model(x_np)

assert np.allclose(preds.numpy(), preds_compiled, atol=1e-5), "Compiled model predictions do not match original model predictions"


In [4]:
from datetime import datetime

x = torch.randint(0, 2, (1_000, 1, 28, 28), dtype=torch.int16)
x_np = x.bool().numpy()

start_time = datetime.now()
preds = model(x)
end_time = datetime.now()
print(f"Original model inference time: {end_time - start_time}")

start_time = datetime.now()
preds_compiled = compiled_model(x_np)
end_time = datetime.now()
print(f"Compiled model inference time: {end_time - start_time}")


Original model inference time: 0:00:10.176780
Compiled model inference time: 0:00:00.002943


In [6]:
x = torch.randint(0, 2, (1_000_000, 1, 28, 28), dtype=torch.int16)
x_np = x.bool().numpy()

start_time = datetime.now()
preds_compiled = compiled_model(x_np)
end_time = datetime.now()
print(f"Compiled model inference time: {end_time - start_time}")


Compiled model inference time: 0:00:02.755774
